In [34]:
import sys
from datetime import date
import pandas as pd
import numpy as np
from jugaad_data.nse import stock_df
import seaborn as sns
import ta
import matplotlib.pyplot as plt

In [35]:
def graph(macd_line, signal_line):
    plt.figure(figsize = (12, 6))
    sns.set_style('dark')
    sns.set_theme('paper')
    sns.lineplot(x = macd_line.index, y = macd_line, label = 'macd_line (SBIN)')
    sns.lineplot(x = signal_line.index, y = signal_line, label = 'signal_line (SBIN)')
    diff = signal_line-macd_line
    # sns.barplot(x = diff.index, y = diff, palette= ['red', 'green'], hue)
    plt.bar(diff.index, diff, color=['green' if val > 0 else 'red' for val in diff], width=1)

In [86]:
def linear(test_data, data):
    df = pd.DataFrame()
    df['P. CLOSE'] = data['PREV. CLOSE']
    df['P. OPEN'] = data['OPEN'].shift(1) 
    df['P. VWAP'] = data['VWAP'].shift(1)
    df['P. LOW'] = data['LOW'].shift(1) 
    df['P. HIGH'] = data['HIGH'].shift(1)
    df['P. NO OF TRADES'] = data['NO OF TRADES'].shift(1)
    df['OPEN'] = data['OPEN']
    df['ONES'] = 1
    df = df.drop(index=0)
    matrix = np.array(df)

    x_matrix = np.roll(matrix, shift=1, axis=1)
    y_matrix = data['CLOSE'].drop(index=0)
    
    x_transpose = x_matrix.transpose()
    x_transpose_x = np.array(x_transpose.dot(x_matrix))
    inverse = np.linalg.inv(x_transpose_x)
    x_transpose_y = np.array(x_transpose.dot(y_matrix))

In [87]:
# ------------------------------------ generation of data frame --------------------------------------------
def generate_dataframe(symbol, today, lastday, train_data):
    # formatting the date
    to_day = int(today[:2])
    to_month = int(today[3:5])
    to_year = int(today[6:10])
    
    la_day = int(lastday[:2])
    la_month = int(lastday[3:5])
    la_year = int(lastday[6:10])
    
    df = pd.DataFrame(stock_df(symbol=symbol, from_date=date(to_year, to_month, to_day), to_date=date(la_year, la_month, la_day), series="EQ"))
    df = df[[ "DATE", "CLOSE", "HIGH", "LOW", "PREV. CLOSE", "VWAP", "NO OF TRADES", "OPEN"]]
    df = df.iloc[::-1]
    df['DATE'] = pd.to_datetime(df['DATE'], format='%d-%m-%Y')
    df['DATE'] = df['DATE'].dt.strftime('%d/%m/%Y')
    if (train_data == "0"):
        df.to_csv("Stocks/"+symbol+".csv", index=False)
    else:
        df.to_csv("Stocks/"+symbol+"_train.csv", index = False)

# ----------------------------------------------- pickle --------------------------------------------------------
def write_pickle(DATA, symbol):
    pd.to_pickle(DATA, symbol + ".pkl")

# -------------------------------------------------------- MAIN -----------------------------------------------------------------------
def main():
    arguments = ["SBIN", "SBIN"]
    lastday = ["31/12/2023", "31/12/2022"]
    today = ["01/01/2023", "01/01/2022"]
    train_data = ["0", "1"]
    for i, argument in enumerate(arguments):
        generate_dataframe(argument, today[i], lastday[i], train_data[i])
    #write_pickle(DATA, "Stocks/" + argument)
    
    df1 = pd.read_csv('Stocks/SBIN.csv')
    train_df1 = pd.read_csv('Stocks/SBIN_train.csv')
    linear(df1, train_df1)

if __name__ == "__main__":
    main()

[[ 5.36759016e-01 -1.11852052e-03  3.03214722e-04 -4.80903793e-04
   1.09059216e-04 -8.21001164e-04 -1.84316047e-07  1.06552541e-03]
 [-1.11852052e-03  7.95213628e-04  2.84874771e-04 -2.99231841e-04
  -2.53126571e-04 -2.72241317e-04  2.59558252e-09 -2.53804172e-04]
 [ 3.03214722e-04  2.84874771e-04  4.88419218e-04  5.23849549e-04
  -6.35958139e-04 -6.43803362e-04 -2.41361534e-09 -1.71059449e-05]
 [-4.80903793e-04 -2.99231841e-04  5.23849549e-04  3.70641313e-03
  -1.83358975e-03 -2.14368321e-03 -3.84991977e-09  5.04852834e-05]
 [ 1.09059216e-04 -2.53126571e-04 -6.35958139e-04 -1.83358975e-03
   1.53119890e-03  1.19283366e-03  1.34739114e-08 -2.09478301e-06]
 [-8.21001164e-04 -2.72241317e-04 -6.43803362e-04 -2.14368321e-03
   1.19283366e-03  1.85699441e-03 -7.20223291e-09  7.59781823e-06]
 [-1.84316047e-07  2.59558252e-09 -2.41361534e-09 -3.84991977e-09
   1.34739114e-08 -7.20223291e-09  1.18498727e-12 -2.45882915e-09]
 [ 1.06552541e-03 -2.53804172e-04 -1.71059449e-05  5.04852834e-05
  -